In [33]:
import time
import subprocess
from os import listdir
from os.path import isfile, join
import sklearn.datasets
import nltk.stem
import numpy as np
import pandas as pd
from stop_words import get_stop_words
import re
import scipy as sp
import PyPDF2
import docx
import sys
import os
import os.path
import string
import docx
import docx2txt
from bs4 import BeautifulSoup

from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
stop_words = get_stop_words('english')
english_stemmer = nltk.stem.SnowballStemmer('english')

In [60]:
def file_operation():
    data = []
    mypath = "Users/sahebsingh/Desktop/Projects/Honeywell/Honeywell" #Path to folder/Directory
    
    list_files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    for k in range(len(list_files)):
        document = docx.Document(list_files[k])
    print (document.paragraphs)

In [3]:
def getPdf(file):
    data = []
    file = open(file, 'rb')
    pdfReader = PyPDF2.PdfFileReader(file)
    page_number = pdfReader.numPages
    for i in range(0, page_number):
        pageObj = pdfReader.getPage(i)
        data.append(pageObj.extractText())
        
    return '\n'.join(data)
    

In [4]:
def getDocx(filename):
    my_text = docx2txt.process(filename)
    
    return np.array(my_text)
    

In [5]:
data1 = getDocx('2.docx')

In [17]:
def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)
    
    

In [26]:
def getText1(filename):
    from docx import Document
    document = Document(filename)
    for para in document.paragraphs:
        print(para.text)

In [7]:
data = getPdf('2.pdf')

In [9]:
def review_to_word(raw_text,remove_stopword=False):
    
    #1 Remove all the HTML marks 
    
    review_text=BeautifulSoup(raw_text,"lxml").get_text()
    
    #2 Remove everything except numbers
    
    review_text=re.sub("[^a-zA-A]"," ",review_text)
    
    #3 Convert them to lower letters and split
    
    words=review_text.lower().split()

    #4 Option of removing stop words if they want to.
    
    if remove_stopword==False:
        stops=set(stop_words)
        words=[w for w in words if not w in stops]
        
    #5 Return Words
    
    return(words)


In [10]:
review_to_word(data2)

['xample',
 'document',
 'applicants',
 'v',
 'ntroduction',
 'document',
 'defines',
 'flight',
 'envelope',
 'aircraft',
 'requirements',
 'referenced',
 'compliance',
 'checklist',
 'certification',
 'programme',
 'boundaries',
 'flight',
 'envelope',
 'will',
 'defined',
 'within',
 'document',
 'speeds',
 'calibrated',
 'airspeeds',
 'requirement',
 'given',
 'knots',
 'stated',
 'otherwise',
 'units',
 'used',
 'metric',
 'units',
 'weights',
 'given',
 'mass',
 'units',
 'kg',
 'formulas',
 'require',
 'force',
 'units',
 'input',
 'therefore',
 'calculated',
 'place',
 'wherever',
 'used',
 'ote',
 'speeds',
 'defined',
 'within',
 'document',
 'used',
 'placards',
 'speed',
 'markings',
 'aeroplane',
 'flight',
 'manual',
 'limitations',
 'load',
 'calculations',
 'need',
 'verified',
 'flight',
 'test',
 'eferences',
 'ist',
 'abbreviations',
 'flight',
 'level',
 'centre',
 'gravity',
 'mean',
 'sea',
 'level',
 'lift',
 'curve',
 'slope',
 'aeroplane',
 'rad',
 'c',
 'maxim

In [11]:
#Taking only Training Data.
train_data = sklearn.datasets.fetch_20newsgroups(subset="train")

print(len(train_data.filenames))
#print(train_data.target_names)                                                    


#Now we will load test data. 
test_data = sklearn.datasets.fetch_20newsgroups(subset = "test")
print(len(test_data.filenames))
#print(test_data.target_names) 

11314
7532


In [4]:
""" Clustering Posts """
#This data is noisy if we run data as it is it will give UnicodeDecodeError.
#For this we will use StemmedTfidfVectorizer.

class StemmedTfidfVectorizer(TfidfVectorizer):
    def build_analyzer(self):
        analyzer = super(TfidfVectorizer, self).build_analyzer()
        return lambda doc: (english_stemmer.stem(w) for w in analyzer(doc))

vectorizer = StemmedTfidfVectorizer(min_df=10, max_df=0.5,
                        stop_words='english', decode_error='ignore'
                                    )

x_transform = vectorizer.fit_transform(train_data.data)#Fitting all the data.
num_samples, num_features = x_transform.shape
print("There are %d number of samples and %d number of features" 
        %(num_samples, num_features))


There are 11314 number of samples and 11771 number of features


In [5]:
""" We will apply K-Means and keep the number of clusters as 50  """
num_clusters = 50  

from sklearn.cluster import KMeans   

#Initializing KMeans.
km = KMeans(n_clusters = num_clusters, init = 'random', n_init = 1,
            verbose = 1, random_state = 1)

#Using KMeans on x_transform.
km.fit_transform(x_transform)


Initialization complete
Iteration  0, inertia 19964.293
Iteration  1, inertia 10691.015
Iteration  2, inertia 10584.101
Iteration  3, inertia 10521.382
Iteration  4, inertia 10480.398
Iteration  5, inertia 10454.625
Iteration  6, inertia 10439.627
Iteration  7, inertia 10431.371
Iteration  8, inertia 10424.447
Iteration  9, inertia 10419.519
Iteration 10, inertia 10416.080
Iteration 11, inertia 10413.788
Iteration 12, inertia 10410.496
Iteration 13, inertia 10408.770
Iteration 14, inertia 10408.041
Iteration 15, inertia 10407.361
Iteration 16, inertia 10407.150
Iteration 17, inertia 10406.969
Iteration 18, inertia 10406.829
Iteration 19, inertia 10406.767
Iteration 20, inertia 10406.736
Iteration 21, inertia 10406.677
Iteration 22, inertia 10406.632
Iteration 23, inertia 10406.581
Iteration 24, inertia 10406.540
Iteration 25, inertia 10406.518
Converged at iteration 25: center shift 0.000000e+00 within tolerance 8.348966e-09


array([[ 1.0234031 ,  1.04555801,  1.03173971, ...,  1.03143964,
         1.07840132,  1.03943592],
       [ 1.02702716,  1.04582618,  1.0366211 , ...,  1.03266984,
         1.07238038,  1.04432137],
       [ 1.01394928,  1.03573708,  1.01364048, ...,  1.01817507,
         1.06918439,  1.03252635],
       ..., 
       [ 1.02611701,  1.05168322,  1.0385245 , ...,  1.03604157,
         1.08018273,  1.0476356 ],
       [ 1.02374697,  1.03828345,  1.02562845, ...,  1.01268084,
         1.07157717,  1.04632214],
       [ 1.02627767,  1.05130621,  1.0383681 , ...,  1.02901612,
         1.0695536 ,  1.04602644]])

In [6]:
post = """Disk drive problems. Hi, I have a problem with my hard disk.
After 1 year it is working only sporadically now.
I tried to format it, but now it doesn't boot any more.
Any ideas? Thanks.
"""

In [7]:
new_post_vec = vectorizer.transform([post])
new_post_label = km.predict(new_post_vec)[0]#We will predict it's cluster 
print(new_post_label)                                            #using km.predcit.
# Comparing all 
similar_indices = (km.labels_ == new_post_label).nonzero()[0]


45


In [8]:
similar = []
for i in similar_indices:
    dist = sp.linalg.norm((new_post_vec - x_transform[i]).toarray())
    similar.append((dist, train_data.data[i]))


In [9]:
similar = sorted(similar) 
print(len(similar))
show_at_1 = similar[0]
show_at_2 = similar[int(len(similar) / 10)]
show_at_3 = similar[int(len(similar) / 2)]

224


In [11]:
print("=== #1 ===")
print(show_at_1)
print()


=== #1 ===
(1.0305214105851774, "From: rogntorb@idt.unit.no (Torbj|rn Rognes)\nSubject: Adding int. hard disk drive to IIcx\nKeywords: Mac IIcx, internal, hard disk drive, SCSI\nReply-To: rogntorb@idt.unit.no (Torbj|rn Rognes)\nOrganization: Div. of CS & Telematics, Norwegian Institute of Technology\nLines: 32\n\nI haven't seen much info about how to add an extra internal disk to a\nmac. We would like to try it, and I wonder if someone had some good\nadvice.\n\nWe have a Mac IIcx with the original internal Quantum 40MB hard disk,\nand an unusable floppy drive. We also have a new spare Connor 40MB\ndisk which we would like to use. The idea is to replace the broken\nfloppy drive with the new hard disk, but there seems to be some\nproblems:\n\nThe internal SCSI cable and power cable inside the cx has only\nconnectors for one single hard disk drive.\n\nIf I made a ribbon cable and a power cable with three connectors each\n(1 for motherboard, 1 for each of the 2 disks), would it work?\n\nIs

In [12]:
print("=== #2 ===")
print(show_at_2)
print()



=== #2 ===
(1.1974156538820808, "From: venaas@flipper.pvv.unit.no (Stig Venaas)\nSubject: Re: CAN'T WRITE TO 720 FLOPPY\nOrganization: ProgramVareVerkstedet - UNIT\nLines: 31\n\nIn article <1qndvd$jhn@usenet.INS.CWRU.Edu> da416@cleveland.Freenet.Edu (Andy Nicola) writes:\n>\n>In a previous article, limagen@hpwala.wal.hp.com () says:\n>\n>>OK all you experts!\n>>Need answer quick.386 machine ,1.44 floppy ; unable to write to a formated\n>>720 disk.Machine claims that disk is write protected,but it is not.\n>>\n>>Note: It 'll read 720's with no problem.\n>>\n>>Please e_mail or post.\n>>\n>>\n>>\n>>\n>If the disk is not an HD-type disk, i.e. with the extra hole in the case\n>opposite the normal write protect hole, the drive will not write to the\n>disk. You can punch a similar hole with whatever is handy or buy a small\n>device, a square hole puncher, for about $19.95...see the back pages of\n>computer shopper magazine for it.\n>\n>To be brief, make the hole any way you can or no writing!

In [13]:
print("=== #3 ===")
print(show_at_3)



=== #3 ===
(1.326820157122635, 'From: cmwand@leland.Stanford.EDU (Christopher Wand)\nSubject: Re: Syquest 150 ???\nOrganization: DSG, Stanford University, CA 94305, USA\nDistribution: usa\nLines: 30\n\nIn article <93759@hydra.gatech.EDU> gt8798a@prism.gatech.EDU (Anthony S. Kim) writes:\n>I remember someone mention about a 150meg syquest.  Has anyone else\n>heard anything about this?  I\'d be interested in the cost per megabyte and the\n>approximate cost of the drive itself and how they compare to the Bernoulli 150.\n\nI think you must be talking about the Syquest 105 (code named Mesa I believe).\nIt is a 3.5" Winchester technology drive pretty much like the other Syquest\ndrives in terms of how it works. According to the latest MacLeak, the \ndrive has a 14.5 ms access time, 1.9 MB/s sustained throughput (these figures\nare from memory so they could be slightly off, but they give you an idea of\nperformance nonetheless). The drive was originally released for the PC\nand just recently 